# Imports & Connections

In [5]:
import pandas as pd
import pyodbc
import sqlite3
import numpy as np

In [6]:
DB = {
    'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',
    'database': 'johari_Herkansing_NEW'
}

ssms_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                     ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Noah\AppData\Local\Temp\ipykernel_108112\2725324115.py:2: SyntaxWarning: invalid escape sequence '\S'
  'servername': 'DESKTOP-IT4OHPV\SQLEXPRESS',


In [7]:
# Local Connections
sqlite_conn = sqlite3.connect('../data/processed/dwh.sqlite')
go_sales_conn = sqlite3.connect('../data/raw/go_sales.sqlite')
go_staff_conn = sqlite3.connect('../data/raw/go_staff.sqlite')
go_crm_conn = sqlite3.connect('../data/raw/go_crm.sqlite')

# Fact tables
fact_tables = ['training', 'satisfaction', 'returned_item', 'orders', 'sales_targetData', 'sales_product_forecast', 'sales_inventory_levels']
fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}

# Dimension tables
dim_tables = ['product', 'sales_staff', 'course', 'satisfaction_type', 'return_reason', 'order_method', 'retailer_contact']
dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}

# Test Output
orders_df = fact_dfs['orders']
product_df = dim_dfs['product']
#product_df.head()

C:\Users\Noah\AppData\Local\Temp\ipykernel_108112\1395309261.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in fact_tables}
C:\Users\Noah\AppData\Local\Temp\ipykernel_108112\1395309261.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_dfs = {table: pd.read_sql(f'SELECT * FROM {table}', ssms_conn) for table in dim_tables}


# Surrogate Key Creation

In [8]:
# Function to add surrogate key to dimension tables
def add_surrogate_key(conn, table_name):
    cursor = conn.cursor()
    # Add SK column
    cursor.execute(f"ALTER TABLE {table_name} ADD SK INT IDENTITY(1,1);")
    # Add EffectiveDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EffectiveDate DATETIME NOT NULL DEFAULT GETDATE();")
    # Add EndDate column
    cursor.execute(f"ALTER TABLE {table_name} ADD EndDate DATETIME NULL;")
    # Add IsActive column
    cursor.execute(f"ALTER TABLE {table_name} ADD IsActive BIT NOT NULL DEFAULT 1;")
    conn.commit()

# Add surrogate keys to dimension tables
#for table in dim_tables:
    #add_surrogate_key(ssms_conn, table)

# Foreign Surrogate Key Creation

In [13]:
# Function to add surrogate foreign key columns to fact tables
def add_surrogate_foreign_key_columns(conn, fact_table, key_mappings):
    cursor = conn.cursor()
    for _, (fact_table, foreign_key_column) in key_mappings.items():
        cursor.execute(f"ALTER TABLE {fact_table} ADD {foreign_key_column} INT;")
    conn.commit()

# Add surrogate foreign key columns to fact tables
for fact_table in fact_tables:
    if fact_table == 'orders':
        key_mappings = {
            #Orders
            #'PRODUCT_id': ('orders', 'PRODUCT_SFK'),
            #'ORDER_METHOD_id': ('orders', 'ORDER_METHOD_SFK'),
            #'RETAILER_CONTACT_id': ('orders', 'RETAILER_CONTACT_SFK'),
            #'RETURNED_REASON': ('orders', 'RETURNED_REASON_SFK'),

            #Sales_targetData
            'PRODUCT_id': ('sales_targetData', 'PRODUCT_SFK'),
            'SALES_STAFF_id': ('sales_targetData', 'SALES_STAFF_SFK'),
            'COURSE_id': ('sales_targetData', 'COURSE_SFK'),
        }
        add_surrogate_foreign_key_columns(ssms_conn, fact_table, key_mappings)

# Function to populate foreign keys in fact tables
def populate_foreign_keys(conn, fact_table, dim_table, natural_key, foreign_key_column, sfk_column):
    cursor = conn.cursor()
    # Adjusted SQL statement to correctly match and update SFKs
    sql = f"""
    UPDATE {fact_table}
    SET {sfk_column} = (
        SELECT {dim_table}.SK
        FROM {dim_table}
        WHERE {dim_table}.{natural_key} = {fact_table}.{foreign_key_column}
    )
    """
    cursor.execute(sql)
    conn.commit()

# Populate Foreign Surrogate Keys

In [14]:
# Populate SFKs for the 'orders' fact table
#populate_foreign_keys(ssms_conn, 'orders', 'product', 'id', 'product_id', 'PRODUCT_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'order_method', 'order_method_id', 'order_method_code', 'ORDER_METHOD_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'RETAILER_CONTACT', 'retailer_id', 'retailer_contact_code', 'RETAILER_CONTACT_SFK')
#populate_foreign_keys(ssms_conn, 'orders', 'RETURN_REASON', 'return_reason_id', '', 'RETURNED_REASON_SFK')

# Populate SFKs for the 'sales_targetData' fact table
populate_foreign_keys(ssms_conn, 'sales_targetData', 'product', 'id', 'product_id', 'PRODUCT_SFK')
populate_foreign_keys(ssms_conn, 'sales_targetData', 'sales_staff', 'sales_staff_id', 'staff_id', 'SALES_STAFF_SFK')
#populate_foreign_keys(ssms_conn, 'sales_targetData', 'course', 'id', 'course_id', 'COURSE_SFK')

# Simulate Slowly Changing Dimensions Type 2

In [ ]:
# Step 1: Identify a Product to Update
# Let's say we're updating the product with ProductID = 'P123'
product_to_update = 'P123'

# Step 2: Insert a New Record for the Updated Product
# Assuming you have a function to generate a new unique SK
new_sk = generate_new_sk()  # This is a placeholder for your SK generation logic
updated_product_info = {
    'ProductID': product_to_update,
    'SK': new_sk,
    'ProductName': 'Updated Product Name',  # Example of an update
    'IsActive': 1,
    'EffectiveDate': datetime.now(),
    # Add other fields as necessary
}
# Append the new record to the product DataFrame
product_df = product_df.append(updated_product_info, ignore_index=True)

# Step 3: Update the IsActive Status of the Old Record
product_df.loc[(product_df['ProductID'] == product_to_update) & (product_df['IsActive'] == 1) & (product_df['SK'] != new_sk), ['IsActive', 'EndDate']] = [0, datetime.now()]

# Step 4: Update the PRODUCT_SFK in the orders Fact Table
# This example assumes you want to update all orders for the product to reference the new product version
orders_df.loc[orders_df['ProductID'] == product_to_update, 'PRODUCT_SFK'] = new_sk

In [ ]:
# Close connections
ssms_conn.close()
sqlite_conn.close()
go_sales_conn.close()
go_staff_conn.close()
go_crm_conn.close()